Although Basenji is unaware of the locations of known genes in the genome, we can go in afterwards and ask what a model predicts for those locations to interpret it as a gene expression prediction.

To do this, you'll need
 * Trained model
 * Gene Transfer Format (GTF) gene annotations
 * BigWig coverage tracks
 * Gene sequences saved in my HDF5 format.
 
First, make sure you have an hg19 FASTA file visible. If you have it already, put a symbolic link into the data directory. Otherwise, I have a machine learning friendly simplified version you can download in the next cell.

In [2]:
import os, subprocess

if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)                

Next, let's grab a few CAGE datasets from FANTOM5 related to heart biology.

These data were processed by
1. Aligning with Bowtie2 with very sensitive alignment parameters.
2. Distributing multi-mapping reads and estimating genomic coverage with bam_cov.py

In [9]:
if not os.path.isfile('data/CNhs11760.bw'):
    subprocess.call('curl -o data/CNhs11760.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs11760.bw', shell=True)
    subprocess.call('curl -o data/CNhs12843.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12843.bw', shell=True)
    subprocess.call('curl -o data/CNhs12856.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12856.bw', shell=True)

Then we'll write out these BigWig files and labels to a samples table.

In [10]:
samples_out = open('data/heart_wigs.txt', 'w')
print('aorta\tdata/CNhs11760.bw', file=samples_out)
print('artery\tdata/CNhs12843.bw', file=samples_out)
print('pulmonic_valve\tdata/CNhs12856.bw', file=samples_out)
samples_out.close()

Predictions in the portion of the genome that we trained might inflate our accuracy, so we'll focus on chr9 genes, which have formed my typical test set. Then we use [basenji_hdf5_genes.py](https://github.com/calico/basenji/blob/master/bin/basenji_hdf5_genes.py) to create the file.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| -l | 262144 | Sequence length. |
| -c | 0.333 | Multiple genes per sequence are allowed, but the TSS must be in the middle 1/3 of the sequence. |
| -p | 3 | Use 3 threads via 
| -t | data/heart_wigs.txt | Save coverage values from this table of BigWig files. |
| -w | 128 | Bin the coverage values at 128 bp resolution. |
| fasta_file | data/hg19.ml.fa | Genome FASTA file for extracting sequences. |
| gtf_file | data/gencode_chr9.gtf | Gene annotations in gene transfer format. |
| hdf5_file | data/gencode_chr9_l262k_w128.h5 | Gene sequence output HDF5 file. |

In [13]:
! basenji_hdf5_genes.py -g data/human.hg19.genome -l 131072 -c 0.333 -p 3 -t data/heart_wigs.txt -w 128 data/hg19.ml.fa data/gencode_chr9.gtf data/gencode_chr9.h5

Now, you can either train your own model in the [Train/test tutorial](https://github.com/calico/basenji/blob/master/tutorials/train_test.ipynb) or download one that I pre-trained.

In [3]:
if not os.path.isdir('models/heart'):
    os.mkdir('models/heart')
if not os.path.isfile('models/heart/model_best.tf.meta'):
    subprocess.call('curl -o models/heart/model_best.tf.index https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.index', shell=True)
    subprocess.call('curl -o models/heart/model_best.tf.meta https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.meta', shell=True)
    subprocess.call('curl -o models/heart/model_best.tf.data-00000-of-00001 https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.data-00000-of-00001', shell=True)

Finally, you can offer data/gencode_chr9_l262k_w128.h5 and the model to [basenji_test_genes.py](https://github.com/calico/basenji/blob/master/bin/basenji_test_genes.py) to make gene expression predictions and benchmark them.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -o | data/gencode_chr9_test | Output directory. |
| --rc | | Average the forward and reverse complement to form prediction. |
| -s | | Make scatter plots, comparing predictions to experiment values. |
| --table | | Print gene expression table. |
| params_file | models/params_small.txt | Table of parameters to setup the model architecture and optimization. |
| model_file | models/gm12878_best.tf | Trained saved model prefix. |
| genes_hdf5_file | data/gencode_chr9_l262k_w128.h5 | HDF5 file containing the gene sequences, annotations, and experiment values. |

In [19]:
! basenji_test_genes.py -o output/gencode_chr9_test --rc -s --table models/params_small.txt models/heart/model_best.tf data/gencode_chr9.h5

/Users/davidkelley/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/davidkelley/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/Users/davidkelley/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
{'num_targets': 3, 'cnn_dilation': [1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'targe

In the output directory *output/gencode_chr9_test/* are several tables and plots describing gene prediction accuracy. For example *gene_cors.txt* contains Spearman and Pearson correlations for predictions versus experimental measurements for all genes and nonzero genes.

In [1]:
! cat output/gencode_chr9_test/gene_cors.txt

0       0.622    0.724    0.652    0.674  aorta 
1       0.563    0.695    0.595    0.575  artery 
2       0.545    0.721    0.661    0.745  pulmonic_valve 


*gene_table.txt.gz* contains specific gene predictions and experimental measurements.

In [5]:
! gunzip -c output/gencode_chr9_test/gene_table.txt.gz | head

ENSG00000277631.4_1   2.906  2.254     0                      
ENSG00000227917.2_1   0.000  0.878     0                      
ENSG00000231808.2_1   0.762  2.418     0                      
ENSG00000170122.5_1   0.000  1.266     0                      
ENSG00000172785.18_2  5.128  4.839     0                      
ENSG00000183784.6_1   0.068  0.257     0                      
ENSG00000235880.1_1   3.355  0.938     0                      
ENSG00000224836.1_1   0.000  0.155     0                      
ENSG00000236594.2_1   0.000  0.320     0                      
ENSG00000225893.1_1   1.054  0.289     0                      


And *gene_scatterX.pdf* plots gene predictions versus experimental measurements for each dataset indexed by *X*.

In [9]:
from IPython.display import IFrame
IFrame('output/gencode_chr9_test/gene_scatter0.pdf', width=600, height=500)